# Tutorial 3: Mechanism Design

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_03_mechanism_design.ipynb)

---

> *"Game theory describes the prison. Mechanism design builds the door. You do not change the players — you change the rules until the prison becomes a marketplace."*
>
> — Brenn Auster, *The Traders Guild Incentive Reform Proposal* (Year 880)

---

## Reverse Game Theory

Game theory analyzes existing games: given the rules, what do rational players do? **Mechanism design** inverts the question: given the outcome you want, what rules produce it?

Tutorial 02 showed that cooperation requires repeated interaction, memory, and a sufficiently long shadow of the future. But what if you could *guarantee* those conditions by design? What if the rules of trade automatically created a game where honesty was the dominant strategy — not because traders are virtuous, but because the mathematics make cheating irrational?

This is Brenn Auster's central contribution: not a theory of how people behave, but a toolkit for engineering systems where self-interested behavior produces cooperative outcomes.

## Learning Objectives

In this tutorial, you will:

1. **Understand mechanism design** — the field of engineering incentive-compatible systems
2. **Implement Vickrey (second-price) auctions** — where truthful bidding is the dominant strategy
3. **Build an incentive compatibility checker** — verify that a mechanism makes honesty rational
4. **Design guild auction mechanics** — apply mechanism design to Traders Guild commodity auctions
5. **Connect to blockchain** — how mechanism design underlies staking, gas auctions, and DeFi

**Technical Concepts:**
- Mechanism design and the Revelation Principle
- Incentive compatibility (DSIC, BIC)
- Vickrey auctions and truthful bidding
- Individual rationality and budget balance

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

guild_trades = pd.read_csv(BASE_URL + "guild_trade_ledger.csv")
game_theory = pd.read_csv(BASE_URL + "game_theory_outcomes.csv")

print(f"Guild trades: {len(guild_trades)} transactions")
print(f"Game theory outcomes: {len(game_theory)} rounds")
print(f"\nCommodities for auction analysis: {guild_trades['commodity'].nunique()}")

## The Three Properties

A mechanism is a set of rules that maps player actions to outcomes. A *good* mechanism satisfies three properties:

1. **Incentive Compatibility (IC)** — Players maximize their payoff by acting truthfully
2. **Individual Rationality (IR)** — No player is worse off participating than abstaining
3. **Budget Balance (BB)** — The mechanism doesn't require outside funding

In [ ]:
# The guild's current auction: first-price sealed-bid
# Problem: bidders shade their bids below true valuation

def first_price_auction(bids, true_values):
    """First-price auction: highest bid wins, pays their bid."""
    winner_idx = np.argmax(bids)
    price = bids[winner_idx]
    winner_value = true_values[winner_idx]
    surplus = winner_value - price  # Winner's profit
    return {
        'winner': winner_idx,
        'price': price,
        'true_value': winner_value,
        'surplus': surplus,
        'revenue': price,
    }

def vickrey_auction(bids, true_values):
    """Vickrey (second-price) auction: highest bid wins, pays second-highest bid."""
    sorted_indices = np.argsort(bids)[::-1]
    winner_idx = sorted_indices[0]
    price = bids[sorted_indices[1]]  # Second-highest bid
    winner_value = true_values[winner_idx]
    surplus = winner_value - price
    return {
        'winner': winner_idx,
        'price': price,
        'true_value': winner_value,
        'surplus': surplus,
        'revenue': price,
    }

# Example: 5 guild members bid on a shipment of manuscript_vellum
np.random.seed(880)
n_bidders = 5
true_values = np.array([120, 95, 110, 80, 130])  # What vellum is truly worth to each
bidders = ['Brenn Auster', 'Torren Gael', 'Tessyn Mord', 'Kellis Vorn', 'Mollen Vek']

print("True valuations:")
for name, val in zip(bidders, true_values):
    print(f"  {name:15s}: {val}")

## First-Price vs. Vickrey: The Incentive Problem

In a first-price auction, rational bidders shade their bids below true value to maximize surplus. In a Vickrey auction, bidding your true value is the dominant strategy.

In [ ]:
# First-price: rational bidders shade bids
shade_factor = 0.75  # Bid 75% of true value
shaded_bids = true_values * shade_factor

result_fp = first_price_auction(shaded_bids, true_values)
print("=== FIRST-PRICE AUCTION (shaded bids) ===")
print(f"Bids: {dict(zip(bidders, shaded_bids.round(1)))}")
print(f"Winner: {bidders[result_fp['winner']]}")
print(f"Price paid: {result_fp['price']:.1f}")
print(f"True value: {result_fp['true_value']}")
print(f"Winner surplus: {result_fp['surplus']:.1f}")
print(f"Seller revenue: {result_fp['revenue']:.1f}")

# Vickrey: truthful bidding is dominant
result_v = vickrey_auction(true_values, true_values)
print(f"\n=== VICKREY AUCTION (truthful bids) ===")
print(f"Bids: {dict(zip(bidders, true_values))}")
print(f"Winner: {bidders[result_v['winner']]}")
print(f"Price paid: {result_v['price']} (second-highest bid)")
print(f"True value: {result_v['true_value']}")
print(f"Winner surplus: {result_v['surplus']}")
print(f"Seller revenue: {result_v['revenue']}")

In [ ]:
# Prove Vickrey is incentive-compatible: 
# Show that no bidder benefits from lying

print("Incentive Compatibility Proof for Vickrey Auction:\n")
print(f"Mollen Vek (true value = {true_values[4]}) considers deviating:\n")

test_bids = [80, 100, 110, 120, 130, 140, 150]
for test_bid in test_bids:
    bids = true_values.copy()
    bids[4] = test_bid  # Mollen Vek's bid
    result = vickrey_auction(bids, true_values)
    
    if result['winner'] == 4:
        utility = true_values[4] - result['price']
    else:
        utility = 0  # Didn't win
    
    marker = ' ← TRUTHFUL' if test_bid == true_values[4] else ''
    print(f"  Bid={test_bid:4d}: winner={bidders[result['winner']]:15s}, "
          f"Vek utility={utility:5.0f}{marker}")

print(f"\nBidding truthfully (130) maximizes Mollen Vek's utility.")
print(f"Overbidding risks winning at a loss. Underbidding risks losing.")
print(f"Truthful bidding is a dominant strategy in Vickrey auctions.")

## The Revelation Principle

The **Revelation Principle** is mechanism design's most powerful theorem: any outcome achievable by any mechanism can also be achieved by a *truthful* mechanism — one where players report their private information honestly.

This means we only need to search among truthful mechanisms. A huge simplification.

In [ ]:
# The Revelation Principle in action: 
# Compare revenue across many first-price auctions (with strategic shading)
# vs Vickrey auctions (with truthful bidding)

np.random.seed(875)
n_auctions = 500
fp_revenues, v_revenues = [], []

for _ in range(n_auctions):
    values = np.random.uniform(50, 150, size=5)
    
    # First-price: bidders shade by random amount (realistic strategic behavior)
    shade_factors = np.random.uniform(0.6, 0.9, size=5)
    fp_bids = values * shade_factors
    fp_result = first_price_auction(fp_bids, values)
    fp_revenues.append(fp_result['revenue'])
    
    # Vickrey: truthful bidding (dominant strategy)
    v_result = vickrey_auction(values, values)
    v_revenues.append(v_result['revenue'])

print(f"Revenue comparison over {n_auctions} auctions:")
print(f"  First-price (shaded): mean={np.mean(fp_revenues):.1f}, std={np.std(fp_revenues):.1f}")
print(f"  Vickrey (truthful):   mean={np.mean(v_revenues):.1f}, std={np.std(v_revenues):.1f}")

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(fp_revenues, bins=30, alpha=0.6, label='First-price (shaded)', color='darkorange', edgecolor='black')
ax.hist(v_revenues, bins=30, alpha=0.6, label='Vickrey (truthful)', color='steelblue', edgecolor='black')
ax.axvline(np.mean(fp_revenues), color='darkorange', linestyle='--', linewidth=2)
ax.axvline(np.mean(v_revenues), color='steelblue', linestyle='--', linewidth=2)
ax.set_xlabel('Auction Revenue')
ax.set_ylabel('Count')
ax.set_title('Revenue Equivalence: First-Price vs. Vickrey')
ax.legend()
plt.tight_layout()
plt.show()

print(f"\nThe Revenue Equivalence Theorem: under certain conditions,")
print(f"all standard auction formats generate the same expected revenue.")
print(f"But Vickrey is SIMPLER — no strategic calculation needed.")

## Incentive Compatibility Checker

Let's build a general tool for checking whether a mechanism is incentive-compatible.

In [ ]:
def check_incentive_compatibility(mechanism_fn, n_players, value_range=(50, 150), n_tests=200):
    """
    Test whether a mechanism is incentive-compatible.
    For each player, check if truthful reporting maximizes utility.
    """
    violations = 0
    total_checks = 0
    violation_details = []
    
    for _ in range(n_tests):
        true_values = np.random.uniform(*value_range, size=n_players)
        
        # Truthful outcome
        truthful_result = mechanism_fn(true_values.copy(), true_values)
        
        for player in range(n_players):
            if truthful_result['winner'] == player:
                truthful_utility = true_values[player] - truthful_result['price']
            else:
                truthful_utility = 0
            
            # Try deviations
            for deviation in np.linspace(value_range[0] * 0.5, value_range[1] * 1.5, 20):
                if abs(deviation - true_values[player]) < 1:
                    continue
                
                deviated_bids = true_values.copy()
                deviated_bids[player] = deviation
                dev_result = mechanism_fn(deviated_bids, true_values)
                
                if dev_result['winner'] == player:
                    dev_utility = true_values[player] - dev_result['price']
                else:
                    dev_utility = 0
                
                total_checks += 1
                if dev_utility > truthful_utility + 0.01:  # Meaningful improvement
                    violations += 1
                    violation_details.append({
                        'player': player, 'true_value': true_values[player],
                        'deviation': deviation, 'truthful_utility': truthful_utility,
                        'deviated_utility': dev_utility
                    })
    
    return violations, total_checks, violation_details

print("Checking incentive compatibility...\n")

v_violations, v_checks, _ = check_incentive_compatibility(vickrey_auction, 5)
print(f"Vickrey auction: {v_violations}/{v_checks} violations ({v_violations/max(v_checks,1):.2%})")

fp_violations, fp_checks, fp_details = check_incentive_compatibility(first_price_auction, 5)
print(f"First-price auction: {fp_violations}/{fp_checks} violations ({fp_violations/max(fp_checks,1):.2%})")

print(f"\nVickrey: incentive-compatible (0 violations)")
print(f"First-price: NOT incentive-compatible (bidders benefit from shading)")

## Guild Commodity Auctions

Let's apply mechanism design to the guild's actual commodity market. We'll use the trade ledger to estimate commodity valuations and design an auction mechanism.

In [ ]:
# Estimate commodity valuations from trade data
commodity_prices = guild_trades.groupby('commodity')['agreed_price'].agg(['mean', 'std', 'count'])
commodity_prices = commodity_prices.sort_values('mean', ascending=False)

print("Commodity price distributions (basis for auction valuations):")
print(f"{'Commodity':20s} {'Mean':>8s} {'Std':>8s} {'Trades':>7s}")
for commodity, row in commodity_prices.iterrows():
    print(f"{commodity:20s} {row['mean']:8.2f} {row['std']:8.2f} {int(row['count']):7d}")

In [ ]:
# Simulate a guild commodity auction day
np.random.seed(880)

# Pick top 6 commodities for auction
auction_commodities = commodity_prices.head(6).index.tolist()
guild_bidders = ['Brenn Auster', 'Torren Gael', 'Tessyn Mord',
                 'Kellis Vorn', 'Mollen Vek', 'Serath Kyne',
                 'Vagabu Olt', 'Quonxy']

auction_results = []
for commodity in auction_commodities:
    mean_price = commodity_prices.loc[commodity, 'mean']
    std_price = commodity_prices.loc[commodity, 'std']
    
    # Each bidder has a private valuation drawn from the price distribution
    valuations = np.random.normal(mean_price, std_price * 0.5, size=len(guild_bidders))
    valuations = np.maximum(valuations, mean_price * 0.3)  # Floor
    
    # Run Vickrey auction
    result = vickrey_auction(valuations, valuations)
    auction_results.append({
        'commodity': commodity,
        'winner': guild_bidders[result['winner']],
        'winning_bid': round(valuations[result['winner']], 2),
        'price_paid': round(result['price'], 2),
        'surplus': round(result['surplus'], 2),
        'revenue': round(result['revenue'], 2),
        'n_bidders': len(guild_bidders)
    })

auction_df = pd.DataFrame(auction_results)
print("Guild Commodity Auction Results (Vickrey mechanism):")
print(auction_df.to_string(index=False))

## The VCG Mechanism

The Vickrey-Clarke-Groves (VCG) mechanism generalizes Vickrey auctions to multi-item settings. Each participant pays the **externality** they impose on others — the reduction in others' welfare caused by their participation.

In [ ]:
def vcg_mechanism(valuations, items_to_allocate=1):
    """
    VCG mechanism: allocate items to highest-value bidders,
    charge each winner the externality they impose.
    """
    n = len(valuations)
    sorted_idx = np.argsort(valuations)[::-1]
    winners = sorted_idx[:items_to_allocate]
    
    results = []
    for w in winners:
        # Total welfare without this player
        others = [v for i, v in enumerate(valuations) if i != w]
        welfare_without = sum(sorted(others, reverse=True)[:items_to_allocate])
        
        # Total welfare of others WITH this player
        other_winners = [i for i in winners if i != w]
        welfare_with = sum(valuations[i] for i in sorted_idx[1:items_to_allocate + 1]
                          if i != w)
        # Payment = welfare_without - welfare_of_others_with
        others_actual = sum(valuations[sorted_idx[j]] 
                           for j in range(items_to_allocate) if sorted_idx[j] != w)
        payment = welfare_without - others_actual
        
        results.append({
            'winner': w,
            'value': valuations[w],
            'payment': max(payment, 0),
            'surplus': valuations[w] - max(payment, 0)
        })
    
    return results

# Multi-item VCG: 3 trade licenses auctioned to 8 bidders
np.random.seed(880)
license_values = np.random.uniform(50, 200, size=8)

print("VCG Auction: 3 Trade Licenses for 8 Guild Members\n")
print("Bidder valuations:")
for i, (name, val) in enumerate(zip(guild_bidders, license_values)):
    print(f"  {name:15s}: {val:.1f}")

vcg_results = vcg_mechanism(license_values, items_to_allocate=3)
print(f"\nWinners:")
total_revenue = 0
for r in vcg_results:
    print(f"  {guild_bidders[r['winner']]:15s}: value={r['value']:.1f}, "
          f"pays={r['payment']:.1f}, surplus={r['surplus']:.1f}")
    total_revenue += r['payment']
print(f"\nTotal revenue: {total_revenue:.1f}")
print(f"Each winner pays their externality — the harm their winning causes to others.")

## Mechanism Design for Blockchain

Every blockchain mechanism is a mechanism design problem:

| Blockchain Mechanism | Desired Outcome | Design Challenge |
|---------------------|-----------------|------------------|
| **Gas auctions** | Efficient block space allocation | EIP-1559 base fee + priority tip |
| **Staking** | Honest validation | Rewards for good behavior, slashing for bad |
| **AMMs** | Fair token exchange | Constant product formula prevents manipulation |
| **Governance voting** | Community preference aggregation | Quadratic voting prevents plutocracy |

In [ ]:
# EIP-1559 gas auction: base fee + priority tip
# The base fee is burned (not paid to validators), removing the incentive to
# create fake congestion. The tip goes to validators.

def eip1559_auction(bids, base_fee):
    """
    EIP-1559 style gas auction.
    Transactions included if max_fee >= base_fee.
    Payment = base_fee + min(priority_tip, max_fee - base_fee).
    """
    results = []
    for i, (max_fee, priority_tip) in enumerate(bids):
        included = max_fee >= base_fee
        if included:
            effective_tip = min(priority_tip, max_fee - base_fee)
            payment = base_fee + effective_tip
        else:
            effective_tip = 0
            payment = 0
        results.append({
            'bidder': i, 'max_fee': max_fee, 'priority_tip': priority_tip,
            'included': included, 'payment': payment,
            'base_fee_burned': base_fee if included else 0,
            'tip_to_validator': effective_tip
        })
    return pd.DataFrame(results)

# Simulate guild trade transactions competing for block space
np.random.seed(42)
n_txs = 12
base_fee = 10
bids = [(np.random.uniform(5, 25), np.random.uniform(1, 5)) for _ in range(n_txs)]

gas_results = eip1559_auction(bids, base_fee)
print(f"EIP-1559 Gas Auction (base_fee = {base_fee}):")
print(f"{'Tx':>3s} {'MaxFee':>7s} {'Tip':>5s} {'Included':>9s} {'Payment':>8s} {'Burned':>7s} {'ToValidator':>12s}")
for _, row in gas_results.iterrows():
    print(f"{int(row['bidder']):3d} {row['max_fee']:7.1f} {row['priority_tip']:5.1f} "
          f"{'YES' if row['included'] else 'no':>9s} {row['payment']:8.1f} "
          f"{row['base_fee_burned']:7.1f} {row['tip_to_validator']:12.1f}")

included = gas_results[gas_results['included']]
print(f"\nIncluded: {len(included)}/{n_txs} transactions")
print(f"Total burned: {included['base_fee_burned'].sum():.1f}")
print(f"Total to validator: {included['tip_to_validator'].sum():.1f}")

In [ ]:
# Why EIP-1559 is better than first-price gas auction:
# 1. Predictable pricing (base fee adjusts algorithmically)
# 2. No overpaying (you can set max_fee = true_urgency)
# 3. Burning removes validator incentive to create fake congestion

# Simulate base fee adjustment over 20 blocks
base_fees = [10.0]
target_capacity = 0.5  # 50% full is the target
max_change = 0.125  # 12.5% max change per block

for block in range(20):
    # Random utilization
    utilization = np.random.beta(2, 2)  # Centered around 50%
    
    # Adjust base fee toward target
    if utilization > target_capacity:
        adjustment = 1 + max_change * (utilization - target_capacity) / target_capacity
    else:
        adjustment = 1 - max_change * (target_capacity - utilization) / target_capacity
    
    new_fee = base_fees[-1] * adjustment
    base_fees.append(max(new_fee, 1.0))  # Floor at 1.0

plt.figure(figsize=(10, 4))
plt.plot(base_fees, 'o-', color='steelblue', linewidth=2)
plt.axhline(y=10, color='gray', linestyle='--', alpha=0.5, label='Initial base fee')
plt.xlabel('Block Number')
plt.ylabel('Base Fee')
plt.title('EIP-1559 Base Fee Adjustment (Algorithmic Price Discovery)')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
print(f"The base fee converges toward equilibrium — no strategic bidding needed.")

## Designing the Guild's Staking Mechanism

With mechanism design tools in hand, let's sketch the staking mechanism that Auster will fully implement in Tutorial 05. The design goal: make honest trading the dominant strategy.

In [ ]:
# Mechanism design for guild staking
# Goal: IC (honesty is dominant), IR (participation is voluntary), BB (self-funding)

def guild_staking_mechanism(stake, trade_value, honest, detection_prob=0.4):
    """
    Calculate payoffs under the staking mechanism.
    """
    reward_rate = 0.05  # 5% reward for honest trading
    slash_rate = 1.0    # Lose entire stake if caught cheating
    cheat_gain = 0.20   # 20% margin from cheating
    
    if honest:
        return trade_value + stake * reward_rate
    else:
        # Expected payoff of cheating
        if_undetected = trade_value * (1 + cheat_gain) + stake * reward_rate
        if_detected = trade_value * (1 - 0.30) - stake * slash_rate
        return (1 - detection_prob) * if_undetected + detection_prob * if_detected

# Check IC: is honesty dominant for all stake levels?
trade_value = guild_trades['total_value'].mean()
stakes = np.linspace(0, trade_value * 2, 50)
honest_payoffs = [guild_staking_mechanism(s, trade_value, True) for s in stakes]
cheat_payoffs = [guild_staking_mechanism(s, trade_value, False) for s in stakes]

# Find the IC threshold
ic_threshold = None
for i, (h, c) in enumerate(zip(honest_payoffs, cheat_payoffs)):
    if h >= c and ic_threshold is None:
        ic_threshold = stakes[i]

plt.figure(figsize=(10, 5))
plt.plot(stakes, honest_payoffs, '-', color='steelblue', linewidth=2, label='Honest trading')
plt.plot(stakes, cheat_payoffs, '--', color='firebrick', linewidth=2, label='Cheating (expected)')
if ic_threshold:
    plt.axvline(x=ic_threshold, color='black', linestyle=':', label=f'IC threshold: {ic_threshold:.0f}')
plt.xlabel('Stake Amount')
plt.ylabel('Expected Payoff')
plt.title('Mechanism Design: Finding the Incentive-Compatible Stake')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Trade value: {trade_value:.0f}")
print(f"IC threshold: stake ≥ {ic_threshold:.0f} makes honesty dominant")
print(f"As % of trade value: {ic_threshold/trade_value:.0%}")

In [ ]:
# Check IR: is participation voluntary (better than abstaining)?
# Abstaining payoff = 0 (no trade, no reward, no risk)
print("Individual Rationality check:")
print(f"  Honest payoff at IC threshold stake: {guild_staking_mechanism(ic_threshold, trade_value, True):.2f}")
print(f"  Abstaining payoff: 0")
print(f"  IR satisfied: {guild_staking_mechanism(ic_threshold, trade_value, True) > 0}")

# Check BB: is the mechanism self-funding?
# Revenue = slashing penalties + trading fees
# Cost = staking rewards
# BB is satisfied if slashing + fees >= rewards on average
fraud_rate = guild_trades['fraud_detected'].mean()
avg_reward = ic_threshold * 0.05  # 5% reward per honest trade
avg_slash = ic_threshold * fraud_rate  # Slash rate * detection rate

print(f"\nBudget Balance check:")
print(f"  Avg reward paid: {avg_reward:.2f} per trade")
print(f"  Avg slash revenue: {avg_slash:.2f} per trade")
print(f"  Needs trading fee of: {max(avg_reward - avg_slash, 0):.2f} to balance")
print(f"\nAll three properties (IC + IR + BB) achievable with proper parameterization.")

## Exercises

### Exercise 1: All-Pay Auction

In an all-pay auction, everyone pays their bid but only the highest bidder wins. Is this mechanism incentive-compatible? What happens to revenue?

In [ ]:
def all_pay_auction(bids, true_values):
    """All-pay auction: everyone pays their bid, highest bidder wins."""
    winner_idx = np.argmax(bids)
    price = bids[winner_idx]
    revenue = sum(bids)  # All bids collected
    surplus = true_values[winner_idx] - bids[winner_idx]  # Winner surplus
    return {'winner': winner_idx, 'price': price, 'true_value': true_values[winner_idx],
            'surplus': surplus, 'revenue': revenue}

# Is it IC? Compare truthful vs strategic bidding
np.random.seed(42)
values = np.array([120, 95, 110, 80, 130])

truthful_result = all_pay_auction(values, values)
print(f"All-Pay with truthful bids:")
print(f"  Winner: bidder {truthful_result['winner']} (value={values[truthful_result['winner']]})")
print(f"  Revenue: {truthful_result['revenue']} (all bids collected!)")
print(f"  Winner surplus: {truthful_result['surplus']}")
print(f"  Losers pay: {sum(values) - values[truthful_result['winner']]} total (for nothing!)")

# Rational response: shade bids dramatically
shaded = values * 0.3  # Bid only 30% of value
shaded_result = all_pay_auction(shaded, values)
print(f"\nAll-Pay with strategic (30%) bids:")
print(f"  Revenue: {shaded_result['revenue']:.0f} (much lower)")
print(f"\nAll-pay is NOT incentive-compatible: bidders shade to minimize losses.")

### Exercise 2: The Myerson Optimal Auction

Myerson showed that the revenue-maximizing auction sets a **reserve price**. Below the reserve, no sale occurs. Implement a Vickrey auction with reserve price and find the optimal reserve.

In [ ]:
def vickrey_with_reserve(bids, true_values, reserve):
    """Vickrey auction with reserve price."""
    eligible = [(i, b) for i, b in enumerate(bids) if b >= reserve]
    if not eligible:
        return {'winner': -1, 'price': 0, 'revenue': 0, 'surplus': 0, 'sold': False}
    eligible.sort(key=lambda x: x[1], reverse=True)
    winner_idx = eligible[0][0]
    price = eligible[1][1] if len(eligible) > 1 else reserve
    price = max(price, reserve)
    return {'winner': winner_idx, 'price': price, 'revenue': price,
            'surplus': true_values[winner_idx] - price, 'sold': True,
            'true_value': true_values[winner_idx]}

# Find optimal reserve price
np.random.seed(875)
reserves = np.linspace(0, 150, 50)
avg_revenues = []

for reserve in reserves:
    revenues = []
    for _ in range(500):
        vals = np.random.uniform(50, 150, size=5)
        result = vickrey_with_reserve(vals, vals, reserve)
        revenues.append(result['revenue'])
    avg_revenues.append(np.mean(revenues))

optimal_reserve = reserves[np.argmax(avg_revenues)]
plt.figure(figsize=(10, 5))
plt.plot(reserves, avg_revenues, 'o-', color='steelblue', markersize=3)
plt.axvline(x=optimal_reserve, color='firebrick', linestyle='--',
            label=f'Optimal reserve: {optimal_reserve:.0f}')
plt.xlabel('Reserve Price')
plt.ylabel('Average Revenue')
plt.title('Finding the Optimal Reserve Price')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Exercise 3: Detection Probability Sensitivity

How does the IC threshold for guild staking change as detection probability varies from 10% to 90%? At what detection probability does a minimal stake (5% of trade value) become sufficient?

In [ ]:
det_probs = np.linspace(0.1, 0.9, 17)
thresholds = []

for dp in det_probs:
    for s in np.linspace(0, trade_value * 3, 200):
        h = guild_staking_mechanism(s, trade_value, True, detection_prob=dp)
        c = guild_staking_mechanism(s, trade_value, False, detection_prob=dp)
        if h >= c:
            thresholds.append(s)
            break
    else:
        thresholds.append(trade_value * 3)

plt.figure(figsize=(10, 5))
plt.plot(det_probs * 100, [t/trade_value*100 for t in thresholds], 'o-',
         color='steelblue', linewidth=2)
plt.axhline(y=5, color='firebrick', linestyle='--', label='5% stake target')
plt.xlabel('Detection Probability (%)')
plt.ylabel('Required Stake (% of Trade Value)')
plt.title('How Detection Probability Affects Required Stake')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Find where 5% stake is sufficient
for dp, th in zip(det_probs, thresholds):
    if th <= trade_value * 0.05:
        print(f"5% stake sufficient at detection probability ≥ {dp:.0%}")
        break

## Summary

In this tutorial, we learned:

1. **Mechanism design** is the engineering discipline of creating incentive-compatible systems
2. **Vickrey auctions** make truthful bidding the dominant strategy — no strategic calculation needed
3. **The Revelation Principle** lets us focus only on truthful mechanisms
4. **Three properties** define a good mechanism: incentive compatibility, individual rationality, budget balance
5. **EIP-1559** is mechanism design applied to gas pricing — algorithmic price discovery replaces strategic bidding

**Key insight:** Mechanism design is the bridge from game theory to implementation. Game theory tells you what rational agents will do. Mechanism design lets you choose *which* game they play. This is exactly what blockchain protocols do: they define the rules so that the Nash equilibrium IS the desired behavior.

---

**Next Tutorial:** Token Standards (ERC-20, ERC-721) — the building blocks for implementing these mechanisms on-chain.

---

> *"I did not change the traders. I changed the market. The traders responded as I predicted — because I designed the predictions into the market."*
>
> — Brenn Auster